In [1]:
from greeter_allocation import *
from register_salesfloor_acclocation import *
from utils import transform_time_inout, create_working_flag, create_remaining_hours, alert_employee_shortage, convert_df_to_emp_view
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# 1. Read Emp Availability table, filter and format Time columns
path="../00_Input/01_Emp_Availability_Initial.xlsx" # TODO: Read from Frontend
df = pd.read_excel(path, names=['Name', 'Responsibility', 'Time in', 'Time out'])
filtered_df = df[~df['Responsibility'].isin(['Technology', 'Office Work']) & ~df['Name'].str.contains('Available')] # Filter out not-required roles and names
filtered_df= transform_time_inout(filtered_df)

# 2. Create Working Flag and Remaining Hours Left
work_status_df = create_working_flag(filtered_df)
work_status_df= work_status_df[work_status_df['Working Flag']==1] # Filter only working hours for every employee
work_status_df= create_remaining_hours(work_status_df, filtered_df)
work_status_df['Start_time'] = pd.to_datetime(work_status_df['Start_time'], format='%H:%M:%S').dt.time
work_status_df['End_time'] = pd.to_datetime(work_status_df['End_time'], format='%H:%M:%S').dt.time



 df.head()
                    Name       Responsibility             Time in  \
0   Andrew (AJ) # Bowlen         Lead Student 2024-09-16 08:30:00   
1  Analuisa Flores Teran  Sales Floor/Cashier 2024-09-16 08:45:00   
5         Finn Broderick  Sales Floor/Cashier 2024-09-16 08:45:00   
6          Ricardo Gomez  Sales Floor/Cashier 2024-09-16 08:45:00   
8         Solymar Kneale  Sales Floor/Cashier 2024-09-16 08:45:00   

             Time out  
0 2024-09-16 14:00:00  
1 2024-09-16 11:00:00  
5 2024-09-16 11:00:00  
6 2024-09-16 12:00:00  
8 2024-09-16 10:30:00  
Columns:
Index(['Name', 'Responsibility', 'Time in', 'Time out'], dtype='object')
First Row:
Name              Andrew (AJ) # Bowlen
Responsibility            Lead Student
Time in            2024-09-16 08:30:00
Time out           2024-09-16 14:00:00
Name: 0, dtype: object
Second Row:
Name              Analuisa Flores Teran
Responsibility      Sales Floor/Cashier
Time in             2024-09-16 08:45:00
Time out            2024

In [ ]:
#1. Read Shift Req table, format Time columns
emp_count_req= pd.read_excel('../00_Input/02_Emp_Count_Requirement.xlsx') # TODO: Read from Frontend
emp_count_req['From_Time'] = pd.to_datetime(emp_count_req['From_Time'], format='%H:%M:%S').dt.time
emp_count_req['To_Time'] = pd.to_datetime(emp_count_req['To_Time'], format='%H:%M:%S').dt.time

# 2. Alert if available employees are insufficient to satisfy the required count
emp_requirements= alert_employee_shortage(work_status_df, emp_count_req)

No shortage of employees for the whole day


In [4]:
greeter_assignment, greeter_shift_done_dict = allocate_greeter(work_status_df, emp_requirements)

Current Period: 07:30:00 to 08:00:00
Current Period Data:
 Empty DataFrame
Columns: [Name, Start_time, End_time, Working Flag, Remaining_hours_left, Priority]
Index: []
Upstairs Greeters Assigned: []
Downstairs Greeters Assigned: []
-----------------------------------------------------------------------------
Current Period: 08:00:00 to 08:30:00
Current Period Data:
 Empty DataFrame
Columns: [Name, Start_time, End_time, Working Flag, Remaining_hours_left, Priority]
Index: []
Upstairs Greeters Assigned: []
Downstairs Greeters Assigned: []
-----------------------------------------------------------------------------
Current Period: 08:30:00 to 09:00:00
Current Period Data:
                    Name Start_time  End_time  Working Flag  \
0  Andrew (AJ) # Bowlen   08:30:00  09:00:00             1   

   Remaining_hours_left  Priority  
0                   5.5         1  
Upstairs Greeters Assigned: []
Downstairs Greeters Assigned: []
----------------------------------------------------------

In [7]:
register_allocation= allocate_register_salesfloor(emp_requirements, work_status_df, greeter_assignment)
final_allocation= pd.merge(greeter_assignment, register_allocation, how='outer', left_on=['From_Time', 'To_Time'], right_on=['From_Time', 'To_Time'])

---------------------------------------------------------------------------------------------
Allocating registers between 07:30:00 to 08:00:00 ...
No Registers Needed
---------------------------------------------------------------------------------------------
Allocating registers between 08:00:00 to 08:30:00 ...
No Registers Needed
---------------------------------------------------------------------------------------------
Allocating registers between 08:30:00 to 09:00:00 ...
No Registers Needed
---------------------------------------------------------------------------------------------
Allocating registers between 09:00:00 to 09:30:00 ...
Total # Emp Working: 8
Total Emp Working: ['Andrew (AJ) # Bowlen', 'Analuisa Flores Teran', 'Finn Broderick', 'Ricardo Gomez', 'Solymar Kneale', 'Carson Turk', 'Dev Mahajan', 'Ethan Palmer']
Greeters Count: 0
Current Greeters: []
RU_retained_count 0
RD_retained_count 0
retained_RUs []
retained_RDs []
new_RU_needed_count 2
new_RD_needed_count 2
ne

In [8]:
# Get the final employee view
emp_view= convert_df_to_emp_view(final_allocation)

# Format time
final_allocation["From_Time"] = final_allocation["From_Time"].dt.strftime("%I:%M %p")
final_allocation["To_Time"] = final_allocation["To_Time"].dt.strftime("%I:%M %p")
final_allocation.drop(columns=['Greeter_Down_Needed', 'Greeter_Up_Needed', 'Reg_Up_Needed', 'Reg_Down_Needed'], inplace=True)

Data sorting and formating Successfull


In [9]:
# Get the current date and time
current_datetime = datetime.now()
current_datetime = current_datetime.strftime("%Y-%m-%d-%H-%M-%S")

# Save for flask output
final_allocation.to_excel(r"..\01_Output\Final_Allocation.xlsx", index=False)
emp_view.to_excel(r"..\01_Output\Final_Allocation_Emp_View.xlsx", index=False)

final_allocation.to_excel(fr"..\01_Output\Historical_Files\Final_Allocation_{current_datetime}.xlsx", index=False)
emp_view.to_excel(fr"..\01_Output\Historical_Files\Final_Allocation_Emp_View_{current_datetime}.xlsx", index=False)